### 1.create s3 bucket
-Boto3 is the Python Software Development Kit (SDK) for Amazon Web Services (AWS), enabling you to create, update, and delete AWS resources directly from your Python code. With Boto3, you can seamlessly integrate AWS services like Amazon S3, EC2, DynamoDB, and more into your Python applications, libraries, or scripts

In [165]:
import boto3

s3 = boto3.resource('s3')


In [166]:
bucket_name ='yahoofinancestockpriceproject1'

try:

    s3.create_bucket (Bucket=bucket_name) 
    print('S3 bucket has been created successfully')

except Exception as e:

    print('S3 error: ' , e)

S3 bucket has been created successfully


### 2.create train and validation csv

In [167]:
!pip install yfinance

import pandas as pd 
from datetime import datetime
import yfinance as yf

#initialize porameters 
start_date = datetime(2019, 1, 1) 
end_date = datetime(2021, 1, 1)

#get the deta

df_data =yf.download("AAPL", start =start_date, end =end_date)

df_data.reset_index(inplace=True)

df_data


[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,38.722500,39.712502,38.557499,39.480000,37.750084,148158800
1,2019-01-03,35.994999,36.430000,35.500000,35.547501,33.989902,365248800
2,2019-01-04,36.132500,37.137501,35.950001,37.064999,35.440891,234428400
3,2019-01-07,37.174999,37.207500,36.474998,36.982498,35.362015,219111200
4,2019-01-08,37.389999,37.955002,37.130001,37.687500,36.036125,164101200
...,...,...,...,...,...,...,...
500,2020-12-24,131.320007,133.460007,131.100006,131.970001,129.189499,54930100
501,2020-12-28,133.990005,137.339996,133.509995,136.690002,133.810074,124486200
502,2020-12-29,138.050003,138.789993,134.339996,134.869995,132.028397,121047300
503,2020-12-30,135.580002,135.990005,133.399994,133.720001,130.902634,96452100


### Extract, Load & Transform

In [168]:
df_data.drop(axis=1, columns=['Adj Close'], inplace=True) 
df_data.drop(axis=1, columns=['Date'], inplace=True)

In [169]:
df_data_features = df_data.iloc[:-1,:]

df_data_features

,Open,High,Low,Close,Volume
0,38.722500,39.712502,38.557499,39.480000,148158800
1,35.994999,36.430000,35.500000,35.547501,365248800
2,36.132500,37.137501,35.950001,37.064999,234428400
3,37.174999,37.207500,36.474998,36.982498,219111200
4,37.389999,37.955002,37.130001,37.687500,164101200
...,...,...,...,...,...
499,132.160004,132.429993,130.779999,130.960007,88223700
500,131.320007,133.460007,131.100006,131.970001,54930100
501,133.990005,137.339996,133.509995,136.690002,124486200
502,138.050003,138.789993,134.339996,134.869995,121047300


In [170]:
df_data_targets = df_data.iloc[1:, 0].rename("Targets")
df_data_targets

1       35.994999
2       36.132500
3       37.174999
4       37.389999
5       37.822498
          ...    
500    131.320007
501    133.990005
502    138.050003
503    135.580002
504    134.080002
Name: Targets, Length: 504, dtype: float64

In [171]:
df_data_features ['Target'] = list(df_data_targets)

first_column = df_data_features.pop('Target')

df_data_features.insert(0, 'Target', first_column)

df_data_final = df_data_features
df_data_final

/tmp/ipykernel_9423/2589993178.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_features ['Target'] = list(df_data_targets)


,Target,Open,High,Low,Close,Volume
0,35.994999,38.722500,39.712502,38.557499,39.480000,148158800
1,36.132500,35.994999,36.430000,35.500000,35.547501,365248800
2,37.174999,36.132500,37.137501,35.950001,37.064999,234428400
3,37.389999,37.174999,37.207500,36.474998,36.982498,219111200
4,37.822498,37.389999,37.955002,37.130001,37.687500,164101200
...,...,...,...,...,...,...
499,131.320007,132.160004,132.429993,130.779999,130.960007,88223700
500,133.990005,131.320007,133.460007,131.100006,131.970001,54930100
501,138.050003,133.990005,137.339996,133.509995,136.690002,124486200
502,135.580002,138.050003,138.789993,134.339996,134.869995,121047300


### Train Test Split

In [172]:
import numpy as np
df_randomized = df_data_final.sample (frac=1, random_state=123)
df_randomized

,Target,Open,High,Low,Close,Volume
429,115.230003,118.330002,118.830002,113.610001,115.540001,184642000
282,78.839996,81.184998,81.495003,80.712502,81.237503,80113600
383,97.264999,95.334999,95.980003,94.705002,95.919998,90257200
477,117.180000,118.639999,118.769997,117.290001,117.339996,73604300
428,118.330002,114.720001,115.930000,112.800003,115.360001,140150100
...,...,...,...,...,...,...
98,45.049999,44.950001,45.134998,44.452499,44.915001,146118800
476,118.639999,117.589996,119.059998,116.809998,118.639999,74113000
322,70.599998,70.000000,72.062500,69.512497,71.762497,194994800
382,95.334999,96.262497,96.317497,94.672501,95.752502,125642800


In [173]:
train_data, test_data = np.split(df_randomized, [int(0.8*len (df_randomized))])

print (train_data. shape, test_data.shape)

(403, 6) (101, 6)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


### Set a path and upload dataset to S3 bucket

In [174]:
import os

prefix = 'xgboost-as-a-built-in-algo'
train_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix, 'train', 'train.csv')
test_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix, 'test', 'test.csv')

print(train_csv_path)
print(test_csv_path)

s3://yahoofinancestockpriceproject1/xgboost-as-a-built-in-algo/train/train.csv
s3://yahoofinancestockpriceproject1/xgboost-as-a-built-in-algo/test/test.csv


In [175]:
train_data.to_csv(train_csv_path, index = False, header = False)
test_data.to_csv (test_csv_path, index = False, header = False)

### 3.Build XgBoost Model 
I'm using the XGBoost built-in algorithm to build an XGBoost training container as shown in the following code example. You can automatically spot the XGBoost
built-in algorithm image URI using the SageMaker image_uris.retrieve API (or the get_image_uri API if using Amazon SageMaker Python SDK version 1). If
you want to ensure if the image_uris.retrieve API finds the correct URI, see Common parameters for built-in algorithms and look up xgboost from the full list of
built-in algorithm image URIs and available regions.
After specifying the XGBoost image URI, you can use the XGBoost container to construct an estimator using the SageMaker Estimator API and initiate a
training job. This XGBoost built-in algorithm mode does not incorporate your own XGBoost training script and runs directly on th
Reference: https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html

In [176]:
import sagemaker
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker. inputs import TrainingInput

### Find a XGBoost image URI and builds an XGBoost container

In [177]:
xgboost_container = image_uris. retrieve ("xgboost", boto3. Session().region_name, "1.2-2")
display (xgboost_container)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.2-2'

### Initialize hyperparameters

*Booster Parameters :

1) #### max_depth  - Maximum depth of a tree. Increasing this value makes the model more complex and likely to be overfit. O indicates no limit. A limit is required when grow_policy=depth-wise.

2) #### eta - Step size shrinkage used in updates to prevent overfitting. After each boosting step, you can directly get the weights of new features. The eta parameter actually shrinks the feature weights to make the boosting process more conservative.

3) #### gamma - Minimum loss reduction required to make a further partition on a leaf node of the tree. The larger, the more conservative the algorithm is.

4) #### min_child_weight - Minimum sum of instance weight (hessian) needed in a child. If the tree partition step results in a leaf node with the sum of instance weight less than min_child_weight, the building process gives up further partitioning. In linear regression models, this simply corresponds to a minimum number of instances needed in each node. The larger the algorithm, the more conservative it is.

5) #### subsample - Subsample ratio of the training instance. Setting it to 0.5 means that XGBoost randomly collects half of the grow trees. This prevents overfitting.

*Learning Task Parameters :

objective - Specifies the learning task and the corresponding learning objective. Examples: reg:logistic, multi:softmax,
(Reference: https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst#learning-task-parameters)

In [178]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
       
        "objective":"reg:squarederror",
        "num_round":"50"}

### Set an output path where the trained model will be saved

In [179]:
#first {} is bucket name
#second {} is prefix
# output folder
output_path = 's3://{}/{}/{}/'.format(bucket_name, prefix, 'output')
print (output_path)

s3://yahoofinancestockpriceproject1/xgboost-as-a-built-in-algo/output/


### Construct a SageMaker estimator that calls the xgboost-container

-Enable the train_use_spot_instances constructor arg - a simple self-explanatory boolean.

-Set the train_max_wait  constructor arg - this is an int arg representing the amount of time you are willing to wait for Spot infrastructure to become available. Some instance types are harder to get at Spot prices and you may have to wait longer. You are not charged for time spent waiting for Spot infrastructure to become available, you're only charged for actual compute time spent once Spot instances have been successfully procured.

-train_max_run - The timeout in seconds for training

In [180]:
estimator = sagemaker.estimator.Estimator(image_uri = xgboost_container,
                                               hyperparameters = hyperparameters,
                                               role = sagemaker.get_execution_role(),
                                               instance_count = 1,
                                               instance_type = 'ml.m4.xlarge',
                                               volume_size = 5, # 5 GB
                                               output_path = output_path,
                                               use_spot_instances = True,
                                               max_run = 300,
                                               max_wait=600)
                                                  

### Define the data type and paths to the training and validation datasets

In [181]:
#content_type = "csv"
train_input = TrainingInput("s3://{}/{}/{}/".format(bucket_name, prefix, 'train'), content_type="csv")
test_input = TrainingInput("s3://{}/{}/{}/".format(bucket_name, prefix, 'test'), content_type="csv") 

### Execute the XgBoost training job 

In [182]:
estimator.fit({'train': train_input , 'validation': test_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-09-06-20-20-04-693


2024-09-06 20:20:04 Starting - Starting the training job...
2024-09-06 20:20:35 Starting - Preparing the instances for training......
2024-09-06 20:21:23 Downloading - Downloading input data...
2024-09-06 20:21:43 Downloading - Downloading the training image........[2024-09-06 20:23:21.110 ip-10-2-194-115.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-09-06:20:23:21:INFO] Imported framework sagemaker_xgboost_container.training
[2024-09-06:20:23:21:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2024-09-06:20:23:21:INFO] No GPUs detected (normal if no gpus installed)
[2024-09-06:20:23:21:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-09-06:20:23:21:INFO] Determined delimiter of CSV input is ','
[2024-09-06:20:23:21:INFO] Determined delimiter of CSV input is ','
[2024-09-06:20:23:21:INFO] Determined delimiter of CSV input is ','
[2024-09-06:20:23:21:INFO] Determined delimiter of CSV input is '

### 4.Deploy trained xgb model as Endpoint
I
1. Environment
> Within SageMaker - Serialization by User<br>
> **Outside SageMaker Serialization by Endpoint**
2. Method to invoke the endpoint
> **API - Single Prediction**<br>
> $3 Bucket Batch Prediction
3. Data type based on method
> **API - JSON**<br>
> s3 Bucket CSV

To host a model through Amazon EC2 using Amazon SageMaker, deploy the model that you trained in Create and Run a Training Job by calling the deploy **method** of the **xgb_model estimator**. 

When you call the deploy method, few key things that you need to specify

**initial_instance_count (int)**- The number of instances to deploy the model.

**instance_type (str)** - The type of instances that you want to operate your deployed model.

**serializer (int)** - Serialize input data of various formats (a NumPy array, list, file, or buffer) to a CSV-formatted string. We use this because the XGBoost algorithm accepts input files in CSV format.

In [183]:
from sagemaker.serializers import CSVSerializer

xgb_predictor =estimator.deploy(initial_instance_count=1, instance_type= 'ml.m4.xlarge', serializer=CSVSerializer())

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-09-06-20-23-51-881
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-09-06-20-23-51-881
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-09-06-20-23-51-881


--------!

In [184]:
xgb_predictor.endpoint_name

'sagemaker-xgboost-2024-09-06-20-23-51-881'

### Make prediction with the use of Endpoints

In [185]:
# initialize parameters
start_date = datetime(2021, 1, 4)
end_date = datetime(2021, 1, 5)
# get the data
df_data = yf.download('AAPL', start = start_date, end = end_date)
df_data.reset_index(inplace=True)
df_data

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,133.520004,133.610001,126.760002,129.410004,126.683441,143301900


In [186]:
df_data.drop(axis=1, columns=['Adj Close'], inplace=True)
df_data.drop(axis=1, columns=['Date'], inplace=True)
data_features_array = df_data.values
data_features_array

array([[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
        1.43301900e+08]])

### Serialize data


### Inference - Serialized Input by Sagemaker Function

In [187]:
Y_pred_Fcn = xgb_predictor.predict(data_features_array) .decode('utf-8')
print(Y_pred_Fcn, type(Y_pred_Fcn))

132.3885040283203
 <class 'str'>


In [188]:
from sagemaker.serializers import CSVSerializer
Serialized_Input_Fcn = CSVSerializer().serialize([[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
1.43301900e+08]])
print(Serialized_Input_Fcn, type(Serialized_Input_Fcn))
Y_pred_Fcn = xgb_predictor.predict(Serialized_Input_Fcn) .decode('utf-8')
Y_pred_Fcn

133.520004,133.610001,126.760002,129.410004,143301900.0 <class 'str'>


'132.3885040283203\n'

### Inference - Serialized Input by built-in function (Lambda function friendly)

In [189]:
Input = [[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02, 1.43301900e+08],
[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02, 1.43301900e+08],
[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02, 1.43301900e+08]]

Serialized_Input = ', '.join(map(str, Input[0]))

print(Serialized_Input, type(Serialized_Input))

Y_pred= xgb_predictor.predict(Serialized_Input).decode('utf-8')

Y_pred

133.520004, 133.610001, 126.760002, 129.410004, 143301900.0 <class 'str'>


'132.3885040283203\n'

### 5. Lambda function handler

In [190]:
import boto3

ENDPOINT_NAME = 'sagemaker-xgboost-2024-09-04-21-56-30-042'

runtime = boto3.client('runtime.sagemaker')

def lambda_handler(event, context):
        inputs=event['data']
        result = []
        for input in inputs:
            serialized_input = ','.join(map (str, input))
            
            response = runtime.invoke_endpoint (EndpointName=ENDPOINT_NAME,
                                                ContentType='text/csv',
                                                Body=serialized_input)
            
            result.append(response [ 'Body' ].read().decode())
        return result



In [191]:
Input_json = { 'data':
[[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02, 1.43301900e+08],
[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02, 1.43301900e+08],
[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02, 1.43301900e+08]
]
}
result = lambda_handler (Input_json, _)
result

['132.3885040283203\n', '132.3885040283203\n', '132.3885040283203\n']

In [ ]:
#importing the requests Library
import requests
#defining the api-endpoint
API_ENDPOINT = "https://4o55mgqu1h.execute-api.us-east-1.amazonaws.com/xgb"
#data to be sent to api
json {"data": [[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02, 1.43301900e+08],
[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02, 1.43301900e+08],
[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02, 1.43301900e+08]]
}
# sending post request and saving response as response object
r = requests.post(url = API_ENDPOINT, json = json)

In [192]:
sagemaker. Session().delete_endpoint (xgb_predictor.endpoint)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2024-09-06-20-23-51-881


In [193]:
bucket_to_delete = boto3.resource('s3'). Bucket (bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '98CMRKFZDYSG71WP',
   'HostId': '9HZAMlULadsHEa5vhs8udpybAWRo4+aFI229Qu1QuEzr4nbVisj7ShIcUTGESUhMfHQeB3huTpxZFnluNkyctzwg/RVGUZwoB4cJZlNeEhs=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '9HZAMlULadsHEa5vhs8udpybAWRo4+aFI229Qu1QuEzr4nbVisj7ShIcUTGESUhMfHQeB3huTpxZFnluNkyctzwg/RVGUZwoB4cJZlNeEhs=',
    'x-amz-request-id': '98CMRKFZDYSG71WP',
    'date': 'Fri, 06 Sep 2024 22:01:46 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-09-04-21-45-59-551/debug-output/index/000000000/000000000030_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-09-06-20-20-04-693/debug-output/index/000000000/000000000010_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-09-06-20-20-04-693/debug-output/